In [1]:
import metadata as m
interp = m.load_interps()

/home/daehyun/codespace/brain-age-prediction/RQ/weight_parser.py:176: RuntimeWarning: invalid value encountered in divide
  arr = arr / (norm + eps)
/home/daehyun/codespace/brain-age-prediction/RQ/weight_parser.py:176: RuntimeWarning: invalid value encountered in divide
  arr = arr / (norm + eps)
/home/daehyun/codespace/brain-age-prediction/RQ/weight_parser.py:176: RuntimeWarning: invalid value encountered in divide
  arr = arr / (norm + eps)


defaultdict(dict,
            {'gradxinput': {'resnet10': [array([[ 1.26121592e-03, -5.95354178e-02,  7.41836802e-02, ...,
                       -7.22275922e-03, -2.02710686e-01,  2.35243130e-02],
                      [ 9.85950206e-04, -1.91811896e-02,  9.68090913e-02, ...,
                       -1.03226459e-03, -1.75720507e-01,  2.81147341e-02],
                      [ 1.06365773e-03, -6.79946799e-02,  4.24734633e-01, ...,
                       -4.01604460e-03, -2.99429707e-02,  1.11233038e-02],
                      ...,
                      [ 1.82806891e-03, -5.17302537e-02,  1.33046812e-01, ...,
                       -1.48957814e-02,  3.42463107e-02,  3.46727638e-02],
                      [ 5.83260965e-04,  2.16307337e-02,  2.01623458e-01, ...,
                       -1.29856264e-02, -8.05920095e-02,  3.65806286e-03],
                      [ 3.34908185e-05,  7.47402649e-03,  1.33088196e-01, ...,
                        4.06048462e-04, -7.85297510e-02, -4.67711786e-04]]),
   

In [1]:
from itertools import combinations

import matplotlib.pyplot as plt
import numpy as np

import weight_parser as wp
import stats as s
import constants as C
import plotting as p
import metadata as m
import utils as u

In [16]:
sim_dict = dict()
for xai_method in C.XAI_METHODS[-1:]:
    sims, mats = [], []
    for model_name in C.MODELS:
        if xai_method == "ig" and model_name == "convnext-base":
            continue
        seeds = [42, 43, 44] if model_name != "convnext-base" else [42, 43]
        mats.extend([wp.Weights(model_name=model_name, seed=seed, xai_method=xai_method, verbose=False).normalize_df()\
                     for seed in seeds])

    print(f"XAI Method: {xai_method.capitalize()}", end=" ")
    loop = list(combinations(mats, r=2))
    print(f"# combinations: {len(loop)}")
    for mat1, mat2 in loop:
        simmat = mat1 @ mat2.T
        sim = np.diag(simmat).mean()
        sims.append(sim)
    sim_dict[xai_method] = sims

XAI Method: Ig # combinations: 66


In [17]:
import json

with open("./similarity.json", mode="r") as f:
    _sim_dict = json.load(fp=f)

In [18]:
sim_dict.update(_sim_dict)

In [20]:
import numpy as np
for xai_method in sim_dict:
    print(f"XAI {xai_method:11s}: {np.mean(sim_dict[xai_method]):.4f}")

XAI ig         : 0.4991
XAI gradxinput : 0.5031
XAI gcam_avg   : 0.1388
XAI gbp        : 0.3261
XAI ggcam      : 0.0940
XAI ggcam_avg  : 0.2649
XAI deeplift   : 0.5029
